In [ ]:
import numpy as np, random, operator, pandas as pd, matplotlib.pyplot as plt


In [ ]:
class City:
  def __init__(self, x, y):
    self.x = x
    self.y = y
  def distance(self, city):
    xDis = abs(self.x - city.x)
    yDis = abs(self.y - city.y)
    distance = np.sqrt((xDis ** 2) + (yDis ** 2))
    return distance
  def __repr__(self):
    return "(" + str(self.x) + "," + str(self.y)+ ")"

jmlKota = 6
cityList = []
for i in range(0,jmlKota):
    cityList.append(City(x=int(random.random() * 200),
        y=int(random.random() * 200)))
print (cityList)

[(20,54), (178,47), (6,139), (48,99), (73,164), (181,195)]


In [ ]:
def createRoute(cityList):
  route = random.sample(cityList, len(cityList))
  return route
route = createRoute(cityList)
print ("rute :", route)

rute : [(6,139), (48,99), (178,47), (73,164), (20,54), (181,195)]


In [ ]:
def initialPopulation(popSize, cityList):
  population = []
  for i in range(0, popSize):
      population.append(createRoute(cityList))
  return population
popSize = 4
pop = initialPopulation(popSize, cityList)
print ("populasi : ", pop)

populasi :  [[(181,195), (48,99), (178,47), (73,164), (20,54), (6,139)], [(73,164), (178,47), (6,139), (20,54), (181,195), (48,99)], [(20,54), (48,99), (73,164), (181,195), (178,47), (6,139)], [(48,99), (178,47), (73,164), (6,139), (181,195), (20,54)]]


In [ ]:
class Fitness:
    def __init__(self, route):
        self.route = route
        self.distance = 0
        self.fitness= 0.0

    def routeDistance(self):
        if self.distance ==0:
            pathDistance = 0
            for i in range(0, len(self.route)):
                fromCity = self.route[i]
                toCity = None
                if i + 1 < len(self.route):
                  toCity = self.route[i + 1]
                else:
                  toCity = self.route[0]
                pathDistance += fromCity.distance(toCity)
            self.distance = pathDistance
        return self.distance

    def routeFitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.routeDistance())
        return self.fitness

fitnessResults = {}
for i in range(0,len(pop)):
    fitnessResults[i] = Fitness(pop[i]).routeFitness()
print("fitness :", fitnessResults)

fitness : {0: 0.0011720060716876232, 1: 0.0011285478549976791, 2: 0.0015054854758521346, 3: 0.0012202725194136565}


In [ ]:
def rankRoutes(population):
    fitnessResults = {}
    for i in range(0,len(population)):
        fitnessResults[i] = Fitness(population[i]).routeFitness()
    return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)
popRanked = rankRoutes(pop)
print("rank Routes :", popRanked)

rank Routes : [(2, 0.0015054854758521346), (3, 0.0012202725194136565), (0, 0.0011720060716876232), (1, 0.0011285478549976791)]


In [ ]:
def selection(popRanked, eliteSize):
    selectionResults = []
    df = pd.DataFrame(np.array(popRanked), columns=["Index","Fitness"])
    df['cum_sum'] = df.Fitness.cumsum()
    df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()

    for i in range(0, eliteSize): selectionResults.append(popRanked[i][0])
    for i in range(0, len(popRanked) - eliteSize): pick = 100*random.random()
    for i in range(0, len(popRanked)):
        if pick <= df.iat[i,3]:
            selectionResults.append(popRanked[i][0])
            break
    return selectionResults
eliteSize = 2
selectionResults = selection(popRanked, eliteSize)
print (" selection results :" , selectionResults)

 selection results : [2, 3, 3]


In [ ]:
def matingPool(population, selectionResults):
      matingpool = []
      for i in range(0, len(selectionResults)):
            index = selectionResults[i]
            matingpool.append(population[index])
      return matingpool
currentGen = pop
matingpool = matingPool(currentGen, selectionResults)
print("matingPool :",matingpool)

matingPool : [[(20,54), (48,99), (73,164), (181,195), (178,47), (6,139)], [(48,99), (178,47), (73,164), (6,139), (181,195), (20,54)], [(48,99), (178,47), (73,164), (6,139), (181,195), (20,54)]]


In [ ]:
def breedPopulation(matingpool, eliteSize):
    children = []
    length = len(matingpool) - eliteSize
    pool = random.sample(matingpool, len(matingpool))

    for i in range(0,eliteSize): children.append(matingpool[i])

    for i in range(0, length):
      child = breed(pool[i], pool[len(matingpool)-i-1])
      children.append(child)
    return children
def breed(parent1, parent2):
    child = []
    childP1 = []
    childP2 = []
    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))
    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)
    for i in range(startGene, endGene):
          childP1.append(parent1[i])
    childP2 = [item for item in parent2 if item not in
childP1]
    child = childP1 + childP2
    return child
children = breedPopulation(matingpool, eliteSize)
print ("chidren :", children)

chidren : [[(20,54), (48,99), (73,164), (181,195), (178,47), (6,139)], [(48,99), (178,47), (73,164), (6,139), (181,195), (20,54)], [(48,99), (178,47), (73,164), (6,139), (181,195), (20,54)]]


In [ ]:
def mutatePopulation(population, mutationRate):
    mutatedPop = []
    for ind in range(0, len(population)):
        mutatedInd = mutate(population[ind],mutationRate)
        mutatedPop.append(mutatedInd)
    return mutatedPop
def mutate(individual, mutationRate):
    for swapped in range(len(individual)):
        if(random.random() < mutationRate):
          swapWith = int(random.random() * len(individual))
          city1 = individual[swapped]
          city2 = individual[swapWith]
          individual[swapped] = city2
          individual[swapWith] = city1

    return individual
mutationRate = 0.01
children = mutatePopulation(children, mutationRate)
nextGeneration = children
print ("nextGeneration :",nextGeneration)

nextGeneration : [[(20,54), (48,99), (73,164), (181,195), (178,47), (6,139)], [(48,99), (178,47), (73,164), (6,139), (181,195), (20,54)], [(48,99), (178,47), (73,164), (6,139), (181,195), (20,54)]]


In [ ]:
def geneticAlgorithm(population, popSize, eliteSize, mutationRate, generations):
    pop = initialPopulation(popSize, population)
    print("Initial distance: " + str(1 / rankRoutes(pop)[0][1]))

    for i in range(0, generations):
          pop = nextGeneration(pop, eliteSize,mutationRate)

    print("Final distance: " + str(1 / rankRoutes(pop)[0][1]))
    bestRouteIndex = rankRoutes(pop)[0][0]
    bestRoute = pop[bestRouteIndex]
    return bestRoute

def nextGeneration(currentGen, eliteSize, mutationRate):
      popRanked = rankRoutes(currentGen)
      selectionResults = selection(popRanked, eliteSize)
      matingpool = matingPool(currentGen,selectionResults)
      children = breedPopulation(matingpool, eliteSize)
      nextGeneration = mutatePopulation(children, mutationRate)
      return nextGeneration
geneticAlgorithm(population=cityList, popSize = 10, eliteSize=2, mutationRate=0.01, generations=500)

Initial distance: 666.6533547851325
Final distance: 601.0586515442609


[(73,164), (181,195), (178,47), (20,54), (48,99), (6,139)]